In [ ]:
df_prc_b = pd.read_pickle("/usr/app/data/probabilities/d_prc_b.pkl")
df_prc_s = pd.read_pickle("/usr/app/data/probabilities/d_prc_s.pkl")
df4 = pd.merge(df_prc_b,df_prc_s, on=["t"], suffixes=("_b", "_s")).loc[:,["t","prc_b", "prc_s"]]
df4.rename(columns={"prc_b":"P_B", "prc_s":"P_S"}, inplace=True)
df4["t"] = df4["t"]/int(60*con.tau)
df4

In [ ]:
df_s = pd.read_pickle("/usr/app/data/tmp/viDFStates.pkl") 
#df_s.index.values.tolist()

df_t = pd.read_pickle("/usr/app/data/tmp/constructTrans.pkl") 


In [ ]:
list(filter(lambda s: (s[0] == '1') & (s[2]=="4") &  (s[-1]=="3"), df_s.index.tolist()))

In [ ]:
df_t[["s_key","d_key","trpstrt","trpln","prc_b","prc_s","s_d_key"]].head()
         #.isin(df_s.index.values.tolist()),["s_key","d_key","trpstrt","trpln","prc_b","prc_s","s_d_key"]]

In [1]:
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fnmatch import fnmatch
import itertools

%load_ext autoreload
%autoreload 2

from src.modules.probabilities import Probabilities
import modules.generator as g
from modules.transition import Transition
from src.modules.solutionAlgorithms import SolutionAlgorithms
from src.modules.app import App
from src.modules import constants as con
from src.modules.analysis import Analysis


In [4]:
T = np.arange(2,3)
trip_max = [2]

app = App()

for p in list(itertools.product(T,trip_max)):
    print(p)
    app.runVI(p[0], p[1])

app.putout()

0it [00:00, ?it/s]

(2, 2)
States


6399it [00:00, 16185.71it/s]
  1%|▏         | 80/6300 [00:00<00:08, 695.65it/s]

Decisions


100%|██████████| 6300/6300 [00:08<00:00, 731.93it/s]


Exo


176it [00:00, 1689.68it/s]

Trans


8647884it [08:09, 17667.54it/s]


Algo


In [ ]:
df = pd.read_pickle("/usr/app/data/tmp/viInputDf.pkl")
df_states = pd.read_pickle("/usr/app/data/tmp/viDFStates.pkl")
#sol = SolutionAlgorithms()
#cont = sol.performStandardVI(df, df_states["s_obj"].to_dict())
print(df.shape)
print(df_states.shape)



In [ ]:
df.loc[(df["d_obj"].apply(lambda d: d.get_x_t() == 1)) & (df["s_obj"].apply(lambda s: (s.get_D() > 0) & (s.get_t() == 1))), ["s_key", "d_key", "s_d_key"]]
#print(df.loc[(df["s_key"]== "(1,5.0,9.5,21.5,0.1,0.1)")])
#print(df_states)

In [ ]:
print(len(df_states[df_states["s_obj"].apply(lambda s: (s.get_isTerminal()) )]))
print(87812-87768 )

In [ ]:
from modules.state import State
g.constructDecisions(State(0,5.0,0.0,9.5,0.1,0.1), g.decisionSpace())

In [ ]:
print(df_states[(~df_states.index.to_series().isin(df["s_key"])) & (df_states["s_obj"].apply(lambda s: (not s.get_isTerminal()) & (s.get_V_TA() == 1) & (s.get_B_L() > 3.9)))])
print(len(df_states[(~df_states.index.to_series().isin(df["s_key"]))]))
print(df.loc[(df["s_key"].apply(lambda k: fnmatch(k, "(1,5.0,1,20.5,0.1,0.1)")))])
df_states.loc['(2,3.0,0,0.0,0.0,0.0)']

In [ ]:
#df_states.head()
df_states.loc['(1,1.0,0,0.0,0.0,0.0)']
df.loc[(df["s_key"].apply(lambda k: fnmatch(k, "(?,1.?,0,0.0,0.0,0.0)")))]

In [ ]:
print(cont["cont"].describe())
cont.loc[(cont.cont.astype(float)<= -2150.220000),["s_key","d_key","cont"]]

cont.loc[(cont["s_key"].apply(lambda k: fnmatch(k, "(0,?.?,0,0.0,0.?,0.1)"))) & \
         (cont["d_key"].apply(lambda k: fnmatch(k, "(0.?,0.?,0)"))) & \
         (cont["cont"] == 0.02),["s_key","d_key"]]

In [ ]:
# (1) Derive Probabilties for transition


np.random.seed(14)
prob = Probabilities()

p = prob.getProbabilities(1*con.tau*60)
#p.loc[p.trpstrt=="n",:].head(20)
#sum(p["p"]) # == 1 !
p

In [ ]:
from src.modules import constants as con
# Construct ex_info data frame for all ts
df_p = pd.DataFrame()

for t in np.arange(0, 1440, con.tau*60):
    df_p = df_p.append(prob.getProbabilities(t))
    
sum(df_p.loc[df_p.t == 1425.0,"p"])

In [ ]:
# (2) Derive state space


import time
start = time.process_time()
# your code here    
sts = g.constructStates()
print(time.process_time() - start)
sts.describe()

In [ ]:
sts.head()
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 2) & (sts["B_L"] >= 40) & (sts["D"] == 0)]) ##120
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 10) & (sts["B_L"] >= 40)]) ##120

In [ ]:
# (3) Derive decision space
from tqdm import tqdm
dec = {}

for s in tqdm(sts["obj"].head(1)):
    dec[s.getKey()] = g.constructDecisions(s)

#print(sts.iloc[0])
#print(dec)
print(pd.DataFrame([x.__str__() for x in dec[sts.loc[0, "obj"].getKey()]]))

In [ ]:
# Perform transitions
np.random.seed(14)

prob = Probabilities()
ex_inf = prob.getProbabilities(300).loc[0,:]
t = Transition(None, sts.loc[0, "obj"], dec[sts.loc[0,"obj"].getKey()][-1], ex_inf["p"], ex_inf[0:3])


print(sts.loc[0, "obj"])
print(dec[sts.loc[0,"obj"].getKey()][-1])
print(ex_inf)
print()
print(t.get_s_d().__str__())

In [ ]:
#df.loc[(df["t"] == 0) & (df["s_obj"].apply(lambda x: x.get_B_L() == 4.2)) & (df["d_obj"].apply(lambda d: d.get_x_trip() == 1.0)),~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]
df.loc[(df["t"] == 0) & (df["d_obj"].apply(lambda d: d.get_x_t() == 1.0)),~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]['trpln'].astype(int).describe()
df.loc[ (df["s_obj"].apply(lambda x: (x.get_D() == 0.0) )) ,~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]

In [ ]:
splitRuntime = pd.DataFrame(columns=["t_state", "d_state", "tr_state", "vi"])
splitRuntime.loc["20"] = ["a", "B", "c", "d"]
splitRuntime

In [ ]:
ls = ["a"]
ls += ["b"]
ls